In [1]:
import copy
import gc
import json

import tempfile
import os

# 检查 tempfile 模块使用的临时文件目录
temp_dir = tempfile.gettempdir()
print("Updated temp directory:", temp_dir)

import sys
import importlib

new_path = '/data/mr423/project/ageClassification/'
if new_path not in sys.path:
    sys.path.insert(0, new_path)

# relaod the scgpt files
import scgpt
print("scgpt location: ", scgpt.__file__)
importlib.reload(scgpt)

Updated temp directory: /data/mr423/tmp
scgpt location:  /data/mr423/project/code/ageClassification/scgpt/__init__.py


<module 'scgpt' from '/data/mr423/project/code/ageClassification/scgpt/__init__.py'>

In [2]:
from pathlib import Path
import shutil
import sys
import time
import traceback
from typing import List, Tuple, Dict, Union, Optional
import warnings
import pandas as pd
# from . import asyn
import pickle
import torch
from anndata import AnnData
import scanpy as sc
import scvi
import seaborn as sns
import numpy as np
import wandb
from scipy.sparse import issparse
import matplotlib.pyplot as plt
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from torchtext.vocab import Vocab
from torchtext._torchtext import (
    Vocab as VocabPybind,
)
from sklearn.metrics import confusion_matrix

sys.path.insert(0, "../")
import scgpt as scg

import argparse
from scgpt.model import TransformerModel, AdversarialDiscriminator
from scgpt.tokenizer import tokenize_and_pad_batch, random_mask_value
from scgpt.loss import (
    masked_mse_loss,
    masked_relative_error,
    criterion_neg_log_bernoulli,
)
from scgpt.tokenizer.gene_tokenizer import GeneVocab
from scgpt.preprocess import Preprocessor
from scgpt import SubsetsBatchSampler
from scgpt.utils import set_seed, category_str2int, eval_scib_metrics

sc.set_figure_params(figsize=(6, 6))
os.environ["KMP_WARNINGS"] = "off"
warnings.filterwarnings('ignore')

os.environ["WANDB_MODE"]= "offline"

Global seed set to 0


In [3]:
######################################################################
# Settings for wandb mentior
######################################################################

hyperparameter_defaults = dict(
    seed=0,
    do_train=True,
    load_model="/data/mr423/project/pre_trained_model/scGPT_human",
    n_bins=101,

    epochs=1, # 50 !!!!!!!!!!!!  test only
    lr=0.001,
    batch_size=32,   # 128 !!!!!!!!!!!!  test only

    layer_size=128, # 128
    nlayers=4,  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
    nhead=8,  # number of heads in nn.MultiheadAttention
    
    dropout=0.0,  # dropout probability

    use_fast_transformer=True,
    pre_norm=False,
    amp=True,  # Automatic Mixed Precision
    freeze = True, #freeze
)

run = wandb.init(
    config=hyperparameter_defaults,
    project="age_clusters-test",
    reinit=True,
    settings=wandb.Settings(start_method="fork"),
)
config = wandb.config
print(config)

set_seed(config.seed)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'seed': 0, 'do_train': True, 'load_model': '/data/mr423/project/pre_trained_model/scGPT_human', 'n_bins': 101, 'epochs': 1, 'lr': 0.001, 'batch_size': 32, 'layer_size': 128, 'nlayers': 4, 'nhead': 8, 'dropout': 0.0, 'use_fast_transformer': True, 'pre_norm': False, 'amp': True, 'freeze': True}


In [4]:
######################################################################
# Settings for input and preprocessing
######################################################################
pad_token = "<pad>"
special_tokens = [pad_token, "<cls>", "<eoc>"]
mask_value = "auto"  # for masked values, now it should always be auto

max_seq_len = 3001
n_bins = config.n_bins

# input/output representation
input_style = "binned"  # "normed_raw", "log1p", or "binned"
input_emb_style = "category"  # "category" or "continuous" or "scaling"
cell_emb_style = "cls"  # "avg-pool" or "w-pool" or "cls"


# settings for training
CLS = True  # celltype classification objective

In [5]:
######################################################################
# Settings for optimizer
######################################################################
lr = config.lr  # TODO: test learning rate ratio between two tasks
batch_size = config.batch_size
eval_batch_size = config.batch_size
epochs = config.epochs
early_stop = 5

In [6]:
######################################################################
# Settings for the model
######################################################################
use_fast_transformer = config.use_fast_transformer
fast_transformer_backend = "flash"  # "linear" or "flash"


embsize = config.layer_size  # embedding dimension
d_hid = config.layer_size  # dimension of the feedforward network in TransformerEncoder
nlayers = config.nlayers  # number of TransformerEncoderLayer in TransformerEncoder
nhead = config.nhead  # number of heads in nn.MultiheadAttention
dropout = config.dropout  # dropout probability

In [7]:
######################################################################
# Validate the settings
######################################################################
# %% validate settings
assert input_style in ["normed_raw", "log1p", "binned"]
# assert output_style in ["normed_raw", "log1p", "binned"]

assert input_emb_style in ["category", "continuous", "scaling"]
if input_style == "binned":
    if input_emb_style == "scaling":
        raise ValueError("input_emb_style `scaling` is not supported for binned input.")
elif input_style == "log1p" or input_style == "normed_raw":
    if input_emb_style == "category":
        raise ValueError(
            "input_emb_style `category` is not supported for log1p or normed_raw input."
        )

if input_emb_style == "category":
    mask_value = n_bins + 1
    pad_value = n_bins  # for padding gene expr values
    n_input_bins = n_bins + 2
else:
    mask_value = -1
    pad_value = -2
    n_input_bins = n_bins

In [8]:
######################################################################
# Settings for the running recording
######################################################################
dataset_name = 'biobank-age-cluster'
save_dir = Path(f"../save/dev_{dataset_name}-{time.strftime('%b%d-%H-%M')}/")
save_dir.mkdir(parents=True, exist_ok=True)

print(f"save to {save_dir}")
logger = scg.logger
scg.utils.add_file_handler(logger, save_dir / "run.log")

save to ../save/dev_biobank-age-cluster-Aug28-14-25


In [9]:
######################################################################
# Data loading
######################################################################
adata = sc.read("/data/mr423/project/data/3-OLINK_data_train_withOutlier_all.h5ad")
adata_test = sc.read("/data/mr423/project/data/3-OLINK_data_test_withOutlier_all.h5ad")

print(adata.shape)
print(adata_test.shape)

adata.obs["batch_id"]  = adata.obs["str_batch"] = "0"
adata_test.obs["batch_id"]  = adata_test.obs["str_batch"] = "1" 

adata.var.set_index(adata.var["gene_name"], inplace=True)
adata_test.var.set_index(adata.var["gene_name"], inplace=True)

data_is_raw = False
filter_gene_by_counts = False
adata_test_raw = adata_test.copy()
adata = adata.concatenate(adata_test, batch_key="str_batch")

# make the batch category column
batch_id_labels = adata.obs["str_batch"].astype("category").cat.codes.values
adata.obs["batch_id"] = batch_id_labels


ageGroup_id_labels = adata.obs["Age_Group"].astype("category").cat.codes.values

# ageGroup_types = adata.obs["Age_Group"].unique()

num_types = len(np.unique(ageGroup_id_labels))
id2type = dict(enumerate(adata.obs["Age_Group"].astype("category").cat.categories))
print(id2type)

adata.obs["ageGroup_id"] = ageGroup_id_labels

adata.var["gene_name"] = adata.var.index.tolist()

(37304, 2919)
(4145, 2919)
{0: '30-40', 1: '40-50', 2: '50-60', 3: '60-70', 4: '70-80'}


In [10]:
adata.obs

,sex,DoB_Year,DoB_Month,DoB_Day,DoB,Date_Attend,age,Age_Group,batch_id,str_batch,ageGroup_id
Id,,,,,,,,,,,
2144829-0,0,1939,1,15,1939-01-15,2007-11-16,68.835044,60-70,0,0,3
3154285-0,0,1945,1,15,1945-01-15,2007-07-20,62.507871,60-70,0,0,3
1679423-0,1,1945,11,15,1945-11-15,2009-05-19,63.507187,60-70,0,0,3
1172610-0,1,1941,12,15,1941-12-15,2009-09-23,67.772758,60-70,0,0,3
4011532-0,1,1954,1,15,1954-01-15,2009-10-20,55.761807,50-60,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...
2503594-1,0,1947,2,15,1947-02-15,2009-08-13,62.491444,60-70,1,1,3
3494250-1,1,1945,12,15,1945-12-15,2009-11-10,63.904175,60-70,1,1,3
5746191-1,1,1951,12,15,1951-12-15,2009-06-26,57.530459,50-60,1,1,2


In [11]:
######################################################################
# The pre-trained model
######################################################################
if config.load_model is not None:
    model_dir = config.load_model
    model_config_file = model_dir + "/args.json"
    model_file = model_dir + "/best_model.pt"
    vocab_file = model_dir + "/vocab.json"

    vocab = GeneVocab.from_file(vocab_file)
    shutil.copy(vocab_file, save_dir / "vocab.json")
    for s in special_tokens:
        if s not in vocab:
            vocab.append_token(s)

    adata.var["id_in_vocab"] = [
        1 if gene in vocab else -1 for gene in adata.var["gene_name"]
    ]
    gene_ids_in_vocab = np.array(adata.var["id_in_vocab"])
    logger.info(
        f"match {np.sum(gene_ids_in_vocab >= 0)}/{len(gene_ids_in_vocab)} genes "
        f"in vocabulary of size {len(vocab)}."
    )
    adata = adata[:, adata.var["id_in_vocab"] >= 0]

    # model
    with open(model_config_file, "r") as f:
        model_configs = json.load(f)
    logger.info(
        f"Resume model from {model_file}, the model args will override the "
        f"config {model_config_file}."
    )
    embsize = model_configs["embsize"]
    nhead = model_configs["nheads"]
    d_hid = model_configs["d_hid"]
    nlayers = model_configs["nlayers"]
    n_layers_cls = model_configs["n_layers_cls"]

    print("\n**** parameters from the pre-trained model ****")
    print(f'layer_size = embsize: {model_configs["embsize"]} = d_hid: {model_configs["d_hid"]}, n_layers: {model_configs["nlayers"]}, nhead: {model_configs["nheads"]}')
    print("**** parameters from the pre-trained model ****\n")

    print("**** actual model parameters ****")
    print(f'layer_size = embsize: {embsize} = d_hid: {d_hid}, n_layers: {nlayers}, nhead: {nhead}')
    print("**** actual model parameters ****\n")

# set up the preprocessor, use the args to config the workflow
preprocessor = Preprocessor(
    use_key="X",  # the key in adata.layers to use as raw data
    filter_gene_by_counts=filter_gene_by_counts,  # step 1
    filter_cell_by_counts=False,  # step 2
    normalize_total=3000,  # 3. whether to normalize the raw data and to what sum
    result_normed_key="X_normed",  # the key in adata.layers to store the normalized data
    log1p=data_is_raw,  # 4. whether to log1p the normalized data
    result_log1p_key="X_log1p",
    subset_hvg=False,  # 5. whether to subset the raw data to highly variable genes
    hvg_flavor="seurat_v3" if data_is_raw else "cell_ranger",
    binning=n_bins,  # 6. whether to bin the raw data and to what number of bins
    result_binned_key="X_binned",  # the key in adata.layers to store the binned data
)


adata_test = adata[adata.obs["str_batch"] == "1"]
adata = adata[adata.obs["str_batch"] == "0"]

preprocessor(adata, batch_key=None)
preprocessor(adata_test, batch_key=None)

scGPT - INFO - match 2895/2919 genes in vocabulary of size 60697.
scGPT - INFO - Resume model from /data/mr423/project/pre_trained_model/scGPT_human/best_model.pt, the model args will override the config /data/mr423/project/pre_trained_model/scGPT_human/args.json.

**** parameters from the pre-trained model ****
layer_size = embsize: 512 = d_hid: 512, n_layers: 12, nhead: 8
**** parameters from the pre-trained model ****

**** actual model parameters ****
layer_size = embsize: 512 = d_hid: 512, n_layers: 12, nhead: 8
**** actual model parameters ****

scGPT - INFO - Normalizing total counts ...
scGPT - INFO - Binning data ...
scGPT - INFO - Normalizing total counts ...
scGPT - INFO - Binning data ...


In [12]:
######################################################################
# Split the data to train and test
######################################################################
input_layer_key = {  # the values of this map coorespond to the keys in preprocessing
    "normed_raw": "X_normed",
    "log1p": "X_normed",
    "binned": "X_binned",
}[input_style]
all_counts = (
    adata.layers[input_layer_key].A
    if issparse(adata.layers[input_layer_key])
    else adata.layers[input_layer_key]
)
genes = adata.var["gene_name"].tolist()

ageGroup_labels = adata.obs["ageGroup_id"].tolist()  # make sure count from 0
ageGroup_labels = np.array(ageGroup_labels)


(
    train_data,
    valid_data,
    train_ageGroup,
    valid_ageGroup,
) = train_test_split(
    all_counts, ageGroup_labels, test_size=0.2, shuffle=True
)


if config.load_model is None:
    vocab = Vocab(
        VocabPybind(genes + special_tokens, None)
    )  # bidirectional lookup [gene <-> int]
vocab.set_default_index(vocab["<pad>"])
gene_ids = np.array(vocab(genes), dtype=int)

In [13]:
######################################################################
# Tokenize the data
######################################################################
tokenized_train = tokenize_and_pad_batch(
    train_data,
    gene_ids,
    max_len=max_seq_len,
    vocab=vocab,
    pad_token=pad_token,
    pad_value=pad_value,
    append_cls=True,  # append <cls> token at the beginning
    include_zero_gene=False,
)
tokenized_valid = tokenize_and_pad_batch(
    valid_data,
    gene_ids,
    max_len=max_seq_len,
    vocab=vocab,
    pad_token=pad_token,
    pad_value=pad_value,
    append_cls=True,
    include_zero_gene=False,
)
logger.info(
    f"train set number of samples: {tokenized_train['genes'].shape[0]}, "
    f"\n\t feature length: {tokenized_train['genes'].shape[1]}"
)
logger.info(
    f"valid set number of samples: {tokenized_valid['genes'].shape[0]}, "
    f"\n\t feature length: {tokenized_valid['genes'].shape[1]}"
)

def prepare_data(sort_seq_batch=False) -> Tuple[Dict[str, torch.Tensor]]:
    input_gene_ids_train, input_gene_ids_valid = (
        tokenized_train["genes"],
        tokenized_valid["genes"],
    )

    input_values_train, input_values_valid = (
        tokenized_train["values"],
        tokenized_valid["values"],
    )

    tensor_ageGroup_train = torch.from_numpy(train_ageGroup).long()
    tensor_ageGroup_valid = torch.from_numpy(valid_ageGroup).long()


    train_data_pt = {
        "gene_ids": input_gene_ids_train,
        "values": input_values_train,
        "ageGroup": tensor_ageGroup_train,
    }
    valid_data_pt = {
        "gene_ids": input_gene_ids_valid,
        "values": input_values_valid,
        "ageGroup": tensor_ageGroup_valid,
    }

    return train_data_pt, valid_data_pt


# dataset
class SeqDataset(Dataset):
    def __init__(self, data: Dict[str, torch.Tensor]):
        self.data = data

    def __len__(self):
        return self.data["gene_ids"].shape[0]

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.data.items()}


# data_loader
def prepare_dataloader(
    data_pt: Dict[str, torch.Tensor],
    batch_size: int,
    shuffle: bool = False,
    intra_domain_shuffle: bool = False,
    drop_last: bool = False,
    num_workers: int = 0,
) -> DataLoader:
    if num_workers == 0:
        num_workers = min(len(os.sched_getaffinity(0)), batch_size // 2)

    dataset = SeqDataset(data_pt)

    data_loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        pin_memory=True,
    )
    return data_loader

scGPT - INFO - train set number of samples: 29843, 
	 feature length: 2896
scGPT - INFO - valid set number of samples: 7461, 
	 feature length: 2896


In [14]:
print(tokenized_train["genes"])

tensor([[60695,  8381,  8385,  ..., 12884, 35421, 60694],
        [60695,  8381,  8385,  ...,  4939, 12884, 35421],
        [60695,  8381,  8385,  ..., 60694, 60694, 60694],
        ...,
        [60695,  8385,  8390,  ..., 60694, 60694, 60694],
        [60695,  8381,  8385,  ..., 35421, 60694, 60694],
        [60695,  8381,  8385,  ..., 60694, 60694, 60694]])


In [15]:
print(tokenized_train["values"])

tensor([[  0.,  63.,  70.,  ...,  39.,  83., 101.],
        [  0.,  78.,  82.,  ...,  36.,   1.,  66.],
        [  0.,  70.,   9.,  ..., 101., 101., 101.],
        ...,
        [  0.,  79.,  92.,  ..., 101., 101., 101.],
        [  0.,  36.,  84.,  ...,  43., 101., 101.],
        [  0.,  48.,  63.,  ..., 101., 101., 101.]])


In [48]:
######################################################################
# Load the model
######################################################################

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ntokens = len(vocab)  # size of vocabulary
model = TransformerModel(
    ntokens,
    embsize,
    nhead,
    d_hid,
    nlayers,
    nlayers_cls=3,
    n_cls=num_types,
    vocab=vocab,
    dropout=dropout,
    pad_token=pad_token,
    pad_value=pad_value,
    do_mvc=False,
    do_dab=False,
    use_batch_labels=False,
    num_batch_labels=False,
    domain_spec_batchnorm=False,
    input_emb_style=input_emb_style,
    n_input_bins=n_input_bins,
    cell_emb_style=cell_emb_style,
    explicit_zero_prob=False,
    use_fast_transformer=use_fast_transformer,
    fast_transformer_backend=fast_transformer_backend,
    pre_norm=config.pre_norm,
)
if config.load_model is not None:
    try:
        model.load_state_dict(torch.load(model_file))
        logger.info(f"Loading all model params from {model_file}")
    except:
        # only load params that are in the model and match the size
        model_dict = model.state_dict()
        pretrained_dict = torch.load(model_file)
        pretrained_dict = {
            k: v
            for k, v in pretrained_dict.items()
            if k in model_dict and v.shape == model_dict[k].shape
        }
        # for k, v in pretrained_dict.items():
        #     logger.info(f"Loading params {k} with shape {v.shape}")

        model_dict.update(pretrained_dict)
        model.load_state_dict(model_dict)

pre_freeze_param_count = sum(dict((p.data_ptr(), p.numel()) for p in model.parameters() if p.requires_grad).values())

# Freeze all pre-decoder weights
for name, para in model.named_parameters():
    # print("-"*20)
    print(f"name: {name}")
    # if config.freeze and "encoder" in name and "transformer_encoder" not in name:
    if config.freeze and "encoder" in name:
        print(f"freezing weights for: {name}")
        para.requires_grad = False

post_freeze_param_count = sum(dict((p.data_ptr(), p.numel()) for p in model.parameters() if p.requires_grad).values())

logger.info(f"Total Pre freeze Params {(pre_freeze_param_count )}")
logger.info(f"Total Post freeze Params {(post_freeze_param_count )}")
wandb.log(
        {
            "info/pre_freeze_param_count": pre_freeze_param_count,
            "info/post_freeze_param_count": post_freeze_param_count,
        },
)

model.to(device)
print(model)
wandb.watch(model)

name: encoder.embedding.weight
freezing weights for: encoder.embedding.weight
name: encoder.enc_norm.weight
freezing weights for: encoder.enc_norm.weight
name: encoder.enc_norm.bias
freezing weights for: encoder.enc_norm.bias
name: value_encoder.embedding.weight
freezing weights for: value_encoder.embedding.weight
name: value_encoder.enc_norm.weight
freezing weights for: value_encoder.enc_norm.weight
name: value_encoder.enc_norm.bias
freezing weights for: value_encoder.enc_norm.bias
name: transformer_encoder.layers.0.self_attn.Wqkv.weight
freezing weights for: transformer_encoder.layers.0.self_attn.Wqkv.weight
name: transformer_encoder.layers.0.self_attn.Wqkv.bias
freezing weights for: transformer_encoder.layers.0.self_attn.Wqkv.bias
name: transformer_encoder.layers.0.self_attn.out_proj.weight
freezing weights for: transformer_encoder.layers.0.self_attn.out_proj.weight
name: transformer_encoder.layers.0.self_attn.out_proj.bias
freezing weights for: transformer_encoder.layers.0.self_att

[]

In [49]:
######################################################################
# Loss function
######################################################################
criterion_cls = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=lr, eps=1e-4 if config.amp else 1e-8
)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 1, gamma=0.9)
scaler = torch.cuda.amp.GradScaler(enabled=config.amp)

In [50]:
######################################################################
# Train the model
######################################################################
def train(model: nn.Module, loader: DataLoader) -> None:
    """
    Train the model for one epoch.
    """
    model.train()

    total_loss = 0
    total_num = 0
    
    all_preds = []
    all_labels = []


    for batch, batch_data in enumerate(loader):
        input_gene_ids = batch_data["gene_ids"].to(device)
        input_values = batch_data["values"].to(device)
        ageGroup = batch_data["ageGroup"].to(device)

        src_key_padding_mask = input_gene_ids.eq(vocab[pad_token])
        with torch.cuda.amp.autocast(enabled=config.amp):
            output_dict = model(
                input_gene_ids,
                input_values,
                src_key_padding_mask=src_key_padding_mask,
                batch_labels=None,
                CLS=CLS,
                CCE=False,
                MVC=False,
                ECS=False,
                do_sample=False,
                #generative_training=False
            )

            loss = 0.0
            loss = criterion_cls(output_dict["cls_output"], ageGroup)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        scaler.step(optimizer)
        scaler.update()

        # with warnings.catch_warnings(record=True) as w:
        #     warnings.filterwarnings("always")
        #     torch.nn.utils.clip_grad_norm_(
        #         model.parameters(),
        #         1.0,
        #         error_if_nonfinite=False if scaler.is_enabled() else True,
        #     )
        #     if len(w) > 0:
        #         logger.warning(
        #             f"Found infinite gradient. This may be caused by the gradient "
        #             f"scaler. The current scale is {scaler.get_scale()}. This warning "
        #             "can be ignored if no longer occurs after autoscaling of the scaler."
        #         )

        total_loss += loss.item() * len(input_gene_ids)
        total_num += len(input_gene_ids)

        # 记录预测值和标签，用于计算二分类指标
        preds = output_dict["cls_output"].argmax(dim=1).cpu().numpy()
        labels = ageGroup.cpu().numpy()

        # 将所有批次的预测值和真实值连接起来
        all_preds.extend(preds)
        all_labels.extend(labels)

    # 计算二分类指标
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score

    # 定义评估指标函数
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    # conf_matrix = confusion_matrix(all_labels, all_preds)
    
    epoch_loss = total_loss / total_num
    epoch_accuracy = accuracy_score(all_labels, all_preds)

    logger.info("-" * 89)
    logger.info(f"Epoch {epoch}/{epochs}")
    
    logger.info(
    f"| train | total_num {total_num} | epoch loss {epoch_loss:5.4f} | precision {precision:5.4f} | " 
    f" recall {recall:5.4f} | f1 {f1:5.4f} | " 
    f" epoch_accuracy {epoch_accuracy:5.4f}")

    wandb.log(
        {

            "train/precision": precision,
            "train/recall": recall,
            "train/f1": f1,
            "train/loss": epoch_loss,
            "train/epoch_accuracy": epoch_accuracy,
            "epoch": epoch,
        },
    )




In [51]:
######################################################################
# Evaluate the model
######################################################################
def evaluate(model: nn.Module, loader: DataLoader, return_raw: bool = False) -> float:
    """
    Evaluate the model on the evaluation data.
    """
    model.eval()

    total_loss = 0.0
    total_num = 0

    all_preds = []
    all_labels = []

    total_truth = 0
    with torch.no_grad():
        for batch_data in loader:
            input_gene_ids = batch_data["gene_ids"].to(device)
            input_values = batch_data["values"].to(device)
            ageGroup = batch_data["ageGroup"].to(device)

            src_key_padding_mask = input_gene_ids.eq(vocab[pad_token])
            with torch.cuda.amp.autocast(enabled=config.amp):
                output_dict = model(
                    input_gene_ids,
                    input_values,
                    src_key_padding_mask=src_key_padding_mask,
                    batch_labels=None,
                    CLS=CLS,  # evaluation does not need CLS or CCE
                    CCE=False,
                    MVC=False,
                    ECS=False,
                    do_sample=False,
                    #generative_training = False,
                )

                loss = criterion_cls(output_dict["cls_output"], ageGroup)
            
            total_loss += loss.item() * len(input_gene_ids)
            total_num += len(input_gene_ids)
            
            # 记录预测值和标签，用于计算二分类指标
            preds = output_dict["cls_output"].argmax(dim=1).cpu().numpy()
            labels = ageGroup.cpu().numpy()

            # 将所有批次的预测值和真实值连接起来
            all_preds.extend(preds)
            all_labels.extend(labels)

    # 计算二分类指标
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

    # 定义评估指标函数
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    f1 = f1_score(all_labels, all_preds, average='weighted')
    # conf_matrix = confusion_matrix(all_labels, all_preds)
    
    epoch_loss = total_loss / total_num
    epoch_accuracy = accuracy_score(all_labels, all_preds)

     
    wandb.log(
        {

            "valid/precision": precision,
            "valid/recall": recall,
            "valid/f1": f1,
            "valid/loss": epoch_loss,
            "valid/epoch_accuracy": epoch_accuracy,
            "epoch": epoch,
        },
    )

    scheduler.step()

    logger.info(
    f"| valid | total_num {total_num} | epoch loss {epoch_loss:5.4f} | precision {precision:5.4f} | " 
    f" recall {recall:5.4f} | f1 {f1:5.4f} | " 
    f" epoch_accuracy {epoch_accuracy:5.4f}")
    logger.info("-" * 89)


    return total_loss / total_num


In [52]:
######################################################################
# 训练和验证循环
######################################################################

best_val_loss = float("inf")
best_avg_bio = 0.0
best_model = None

logger.info("Apply the model on the age of the clusters \n")

for epoch in range(1, epochs + 1):

    train_data_pt, valid_data_pt = prepare_data()
    train_loader = prepare_dataloader(
        train_data_pt,
        batch_size=batch_size,
        shuffle=False,
        intra_domain_shuffle=True,
        drop_last=False,
    )
    valid_loader = prepare_dataloader(
        valid_data_pt,
        batch_size=eval_batch_size,
        shuffle=False,
        intra_domain_shuffle=False,
        drop_last=False,
    )

    if config.do_train:
        train(model,loader=train_loader,)
   
    val_loss = evaluate(model,loader=valid_loader)

    # early stop
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)
        best_model_epoch = epoch
        logger.info(f"Best model with score {best_val_loss:5.4f}")
        patience = 0
    else:
        patience += 1
        if patience >= early_stop:
            logger.info(f"Early stop at epoch {epoch}")
            break


# save the model into the save_dir
torch.save(best_model.state_dict(), save_dir / "model.pt")

scGPT - INFO - Apply the model on the age of the clusters 

scGPT - INFO - -----------------------------------------------------------------------------------------
scGPT - INFO - Epoch 1/1
scGPT - INFO - | train | total_num 29843 | epoch loss 1.1127 | precision 0.3495 |  recall 0.4296 | f1 0.3171 |  epoch_accuracy 0.4296
scGPT - INFO - | valid | total_num 7461 | epoch loss 1.1026 | precision 0.1923 |  recall 0.4385 | f1 0.2674 |  epoch_accuracy 0.4385
scGPT - INFO - -----------------------------------------------------------------------------------------
scGPT - INFO - Best model with score 1.1026


In [53]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

######################################################################
# Test the model
######################################################################
def test(model: nn.Module, adata: DataLoader):
    
    all_counts = (
        adata.layers[input_layer_key].A
        if issparse(adata.layers[input_layer_key])
        else adata.layers[input_layer_key]
    )

    # print(adata.layers[input_layer_key])

    ageGroup_labels = adata.obs["ageGroup_id"].tolist()
    ageGroup_labels = np.array(ageGroup_labels)


    tokenized_test = tokenize_and_pad_batch(
        all_counts,
        gene_ids,
        max_len=max_seq_len,
        vocab=vocab,
        pad_token=pad_token,
        pad_value=pad_value,
        append_cls=True,  # append <cls> token at the beginning
        include_zero_gene=False,
    )

    tensor_age_test = torch.from_numpy(ageGroup_labels).long()


    test_data_pt = {
        "gene_ids": tokenized_test["genes"],
        "values": tokenized_test["values"],
        "ageGroup": tensor_age_test,
    }

    test_loader = DataLoader(
        dataset=SeqDataset(test_data_pt),
        batch_size=eval_batch_size,
        shuffle=False,
        drop_last=False,
        num_workers=min(len(os.sched_getaffinity(0)), eval_batch_size // 2),
        pin_memory=True,
    )


    model.eval()

    total_loss = 0.0
    total_num = 0

    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for batch_data in test_loader:
            input_gene_ids = batch_data["gene_ids"].to(device)
            input_values = batch_data["values"].to(device)
            ageGroup = batch_data["ageGroup"].to(device)

            src_key_padding_mask = input_gene_ids.eq(vocab[pad_token])
            with torch.cuda.amp.autocast(enabled=config.amp):
                output_dict = model(
                    input_gene_ids,
                    input_values,
                    src_key_padding_mask=src_key_padding_mask,
                    batch_labels=None,
                    CLS=True, 
                    CCE=False,
                    MVC=False,
                    ECS=False,
                    do_sample=False,
                    #generative_training = False,
                )
                    
                loss = criterion_cls(output_dict["cls_output"], ageGroup)
            
            total_loss += loss.item() * len(input_gene_ids)
            total_num += len(input_gene_ids)
                   
            # 记录预测值和标签，用于计算二分类指标
            preds = output_dict["cls_output"].argmax(dim=1).cpu().numpy()
            targets = ageGroup.cpu().numpy()

            # 将所有批次的预测值和真实值连接起来
            all_preds.extend(preds)
            all_targets.extend(targets)

        # 计算二分类指标
        all_preds = np.array(all_preds)
        all_targets = np.array(all_targets)
    
    
    # 将 Tensor 转换为 Python 列表
    all_preds_list = all_preds.tolist()
    all_targets_list = all_targets.tolist()




    # 定义评估指标函数
    precision = precision_score(all_targets_list, all_preds_list, average='weighted')
    recall = recall_score(all_targets_list, all_preds_list, average='weighted')
    f1 = f1_score(all_targets_list, all_preds_list, average='weighted')
    accuracy = accuracy_score(all_targets_list, all_preds_list)

    logger.info("-" * 89)
    logger.info("-" * 89)
    
    logger.info(
    f"| valid | total_num {total_num} | weighted precision {precision:5.4f} | " 
    f" weighted recall {recall:5.4f} | weighted f1 {f1:5.4f} | " 
    f" weighted accuracy {accuracy:5.4f}")
    logger.info("-" * 89)

    # save to dataFrame
    all_preds_save_dir = str(save_dir) + "/all_preds.txt"
    with open(all_preds_save_dir, "w") as file:
        file.write(str(all_preds_list))
    
    all_targets_save_dir = str(save_dir) + "/all_targets.txt"
    with open(all_targets_save_dir, "w") as file:
        file.write(str(all_targets_list))

    # 创建 DataFrame
    all_preds_df = pd.DataFrame(all_preds_list, columns=['Predictions'])
    all_targets_df = pd.DataFrame(all_targets_list, columns=['Target'])

    # 合并 DataFrame
    combined_df = pd.concat([all_preds_df, all_targets_df], axis=1)

    combined_df['Predictions'] = combined_df['Predictions'].round(1)
    combined_df['Target'] = combined_df['Target'].round(1)

    # 文件路径
    data_save_dir = str(save_dir) + "/output.csv"

    # 保存数据到 CSV 文件
    combined_df.to_csv(data_save_dir, index=False)

    
    
test(model, adata_test)


df = pd.read_csv(str(save_dir) + "/output.csv")

# 提取预测值和实际值
y_pred = df['Predictions']
y_true = df['Target']



# 定义评估指标函数
precision = precision_score(y_true, y_pred, average=None)
recall = recall_score(y_true, y_pred, average=None)
f1 = f1_score(y_true, y_pred, average=None)
accuracy = accuracy_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

# 输出结果
print(f'precision: {precision}')
print(f'recall: {recall}')
print(f'f1: {f1}')
print(f'accuracy: {accuracy:.2f}')
print(f'conf_matrix: {conf_matrix}')

wandb.finish()

scGPT - INFO - -----------------------------------------------------------------------------------------
scGPT - INFO - -----------------------------------------------------------------------------------------
scGPT - INFO - | valid | total_num 4145 | weighted precision 0.2009 |  weighted recall 0.4483 | weighted f1 0.2775 |  weighted accuracy 0.4483
scGPT - INFO - -----------------------------------------------------------------------------------------
precision: [0.        0.        0.4482509 0.       ]
recall: [0. 0. 1. 0.]
f1: [0.         0.         0.61902382 0.        ]
accuracy: 0.45
conf_matrix: [[   0    0  942    0]
 [   0    0 1329    0]
 [   0    0 1858    0]
 [   0    0   16    0]]


epoch,▁▁
info/post_freeze_param_count,▁
info/pre_freeze_param_count,▁
train/epoch_accuracy,▁
train/f1,▁
train/loss,▁
train/precision,▁
train/recall,▁
valid/epoch_accuracy,▁
valid/f1,▁
valid/loss,▁
